# Notebook 07: App Interface Testing

Validate the allergen detection pipeline through an app-like interface simulation. This notebook focuses on request/response shaping, latency checks, error handling, and UI-facing payloads.

## Prerequisites

- Trained NER model available in `models/ner_model/`
- OCR engine dependencies installed (`easyocr`, `opencv-python`)
- Supporting files: `data/allergen_dictionary.json`, `data/ner_training/label_mapping.json`
- Recommended: Run Notebook 06 once to verify pipeline

Run cells in order. This notebook returns UI-ready JSON payloads.

## Section A: Setup and Environment

In [ ]:
# Step 1: Setup paths and environment
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

ROOT = Path.cwd()
if ROOT.name == "notebooks":
    ROOT = ROOT.parent
SRC = ROOT / "src"
DATA_DIR = ROOT / "data"
MODELS_DIR = ROOT / "models"
RESULTS_DIR = ROOT / "results"

if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

print("✓ Paths configured")
print(f"Root: {ROOT}")
print(f"Data: {DATA_DIR}")
print(f"Models: {MODELS_DIR}")

In [ ]:
# Step 2: Imports
import json
import time
from typing import List, Dict, Tuple
from collections import defaultdict

import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
import cv2
from PIL import Image

from ocr.simple_ocr_engine import SimpleOCREngine

print("✓ Imports loaded")
print(f"PyTorch: {torch.__version__}, CUDA: {torch.cuda.is_available()}")

## Section B: Load Pipeline Components

In [ ]:
# Step 3: Load allergen dictionary and label mapping
allergen_path = DATA_DIR / "allergen_dictionary.json"
label_mapping_path = DATA_DIR / "ner_training" / "label_mapping.json"

with open(allergen_path, 'r') as f:
    allergen_dictionary = json.load(f)
with open(label_mapping_path, 'r') as f:
    label_mapping = json.load(f)

id2label = {int(k): v for k, v in label_mapping["id2label"].items()}
label2id = {v: int(k) for k, v in label_mapping["label2id"].items()}

print(f"✓ Allergen dictionary loaded: {len(allergen_dictionary)} types")
print(f"✓ Labels loaded: {list(id2label.values())}")

In [ ]:
# Step 4: Load trained NER model and tokenizer
model_path = MODELS_DIR / "ner_model"
if not model_path.exists():
    experiments = list((MODELS_DIR / "experiments").glob("**/pytorch_model.bin"))
    if experiments:
        model_path = experiments[0].parent
        print(f"ℹ️  Using model from experiments: {model_path}")
    else:
        raise FileNotFoundError("No trained model found. Run Notebook 04 first.")

try:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForTokenClassification.from_pretrained(model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device).eval()
    print(f"✓ Model loaded on device: {device}")
except Exception as e:
    raise RuntimeError(f"Failed to load model: {e}")

In [ ]:
# Step 5: Initialize OCR engine
try:
    ocr_engine = SimpleOCREngine(lang_list=["en"], gpu=torch.cuda.is_available())
    print("✓ OCR engine initialized")
except Exception as e:
    ocr_engine = None
    print(f"⚠️  OCR unavailable: {e}")

## Section C: API-Style Pipeline Functions

In [ ]:
# Step 6: Helper functions (text cleaning, NER, mapping, response formatting)

def clean_text(text: str) -> str:
    if not text:
        return ""
    text = ' '.join(text.split())
    text = text.replace('_', '')
    return text.strip()


def run_ner_prediction(text: str) -> List[Tuple[str, str, float]]:
    if not text or len(text.strip()) < 3:
        return []
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512,
        padding=True,
        return_offsets_mapping=True
    )
    offsets = inputs.pop("offset_mapping")[0].numpy()
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits[0]
        probs = torch.softmax(logits, dim=-1)
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        confs = probs.max(dim=-1).values.cpu().numpy()
    
    entities = []
    current = None
    conf_list = []
    start_idx = 0
    for pred, conf, (s, e) in zip(preds, confs, offsets):
        if s == e:
            continue
        label = id2label[pred]
        if label.startswith('B-'):
            if current:
                entities.append((current, curr_label, float(np.mean(conf_list))))
            current = text[s:e]
            curr_label = label[2:]
            conf_list = [conf]
            start_idx = s
        elif label.startswith('I-') and current:
            current = text[start_idx:e]
            conf_list.append(conf)
        else:
            if current:
                entities.append((current, curr_label, float(np.mean(conf_list))))
                current = None
                conf_list = []
    if current:
        entities.append((current, curr_label, float(np.mean(conf_list))))
    return entities


def map_to_standard_allergens(entities: List[Tuple[str, str, float]]) -> Dict:
    detected = defaultdict(list)
    for text_span, label, conf in entities:
        span_lower = text_span.lower().strip()
        matched = False
        for allergen_type, synonyms in allergen_dictionary.items():
            for syn in synonyms:
                if syn.lower() in span_lower or span_lower in syn.lower():
                    detected[allergen_type].append({
                        'text': text_span,
                        'label': label,
                        'confidence': conf
                    })
                    matched = True
                    break
            if matched:
                break
        if not matched and label != 'O':
            detected['unknown'].append({
                'text': text_span,
                'label': label,
                'confidence': conf
            })
    return dict(detected)


def format_response(result: Dict) -> Dict:
    return {
        'image_path': result.get('image_path'),
        'success': result.get('success', False),
        'error': result.get('error'),
        'raw_text': result.get('raw_text', ''),
        'cleaned_text': result.get('cleaned_text', ''),
        'detected_allergens': result.get('detected_allergens', {}),
        'avg_confidence': result.get('avg_confidence', 0.0),
        'timings': result.get('timings', {}),
        'entities_found': result.get('entities_found', [])
    }

print("✓ Helper functions ready")

In [ ]:
# Step 7: API-style detection function

def detect_allergens_api(request: Dict) -> Dict:
    """
    Simulate an app/API call.
    Expected request fields:
      - image_path: path to image (str)
      - use_ocr: bool (default True)
      - provided_text: optional str (used if use_ocr=False or as fallback)
    Returns UI-ready JSON payload.
    """
    image_path = request.get('image_path')
    use_ocr = request.get('use_ocr', True)
    provided_text = request.get('provided_text')
    verbose = request.get('verbose', False)
    
    resp = {
        'image_path': image_path,
        'success': False,
        'error': None,
        'raw_text': '',
        'cleaned_text': '',
        'detected_allergens': {},
        'avg_confidence': 0.0,
        'entities_found': [],
        'timings': {},
    }
    try:
        if not image_path:
            raise ValueError('image_path is required')
        
        # Step A: OCR or provided text
        t0 = time.time()
        if use_ocr:
            if ocr_engine is None:
                raise RuntimeError('OCR engine not available')
            img = cv2.imread(str(image_path))
            if img is None:
                raise ValueError(f"Cannot read image: {image_path}")
            raw_text = ocr_engine.extract(img)
        else:
            if not provided_text:
                raise ValueError('provided_text is required when use_ocr=False')
            raw_text = provided_text
        resp['timings']['ocr'] = time.time() - t0
        resp['raw_text'] = raw_text
        
        # Step B: Clean text
        t0 = time.time()
        cleaned = clean_text(raw_text)
        resp['cleaned_text'] = cleaned
        resp['timings']['cleaning'] = time.time() - t0
        if not cleaned:
            raise ValueError('No text after cleaning')
        
        # Step C: NER
        t0 = time.time()
        entities = run_ner_prediction(cleaned)
        resp['entities_found'] = entities
        resp['timings']['ner'] = time.time() - t0
        
        # Step D: Mapping
        t0 = time.time()
        mapped = map_to_standard_allergens(entities)
        resp['detected_allergens'] = mapped
        resp['timings']['mapping'] = time.time() - t0
        
        # Step E: Confidence
        confs = [d['confidence'] for v in mapped.values() for d in v]
        resp['avg_confidence'] = float(np.mean(confs)) if confs else 0.0
        resp['success'] = True
        resp['timings']['total'] = sum(resp['timings'].values())
        
        if verbose:
            print(f"✓ {image_path} | allergens: {list(mapped.keys())} | conf: {resp['avg_confidence']:.2f}")
    except Exception as e:
        resp['error'] = str(e)
        if verbose:
            print(f"❌ {image_path}: {e}")
    return resp

print("✓ API-style detection function ready")

## Section D: UI Scenario Tests

In [ ]:
# Step 8: Load test images and optional ground-truth texts

test_samples_dir = DATA_DIR / "ocr_results" / "test_samples"
raw_dir = DATA_DIR / "raw"

test_images = []
if test_samples_dir.exists():
    test_images.extend(list(test_samples_dir.glob('*.jpg')))
    test_images.extend(list(test_samples_dir.glob('*.png')))
if len(test_images) < 5 and raw_dir.exists():
    test_images.extend(list(raw_dir.glob('*.jpg'))[:10])

# deduplicate
test_images = list({p.name: p for p in test_images}.values())

print(f"Found {len(test_images)} test images")
for p in test_images[:5]:
    print(f" - {p.name}")

In [ ]:
# Step 9: Simulate API calls for UI responses

ui_responses = []
max_samples = min(10, len(test_images))

for img_path in test_images[:max_samples]:
    txt_path = img_path.with_suffix('.txt')
    provided_text = None
    if txt_path.exists():
        with open(txt_path, 'r', encoding='utf-8') as f:
            provided_text = f.read()
    
    request = {
        'image_path': str(img_path),
        'use_ocr': ocr_engine is not None,
        'provided_text': provided_text,
        'verbose': True
    }
    resp = detect_allergens_api(request)
    ui_responses.append(resp)

print(f"\n✓ Completed {len(ui_responses)} simulated calls")
success_count = sum(1 for r in ui_responses if r['success'])
print(f"Success rate: {success_count}/{len(ui_responses)}")

## Section E: Response Validation and UX Checks

In [ ]:
# Step 10: Validate response schema and preview

required_keys = {"image_path", "success", "error", "detected_allergens", "avg_confidence", "timings"}

schema_issues = []
for resp in ui_responses:
    missing = required_keys - set(resp.keys())
    if missing:
        schema_issues.append((resp.get('image_path'), list(missing)))

if schema_issues:
    print("⚠️ Schema issues found:")
    for img, miss in schema_issues:
        print(f" - {img}: missing {miss}")
else:
    print("✓ All responses conform to schema")

# Preview first 3 responses
for resp in ui_responses[:3]:
    print("\n--- Response Preview ---")
    print(f"Image: {resp['image_path']}")
    print(f"Success: {resp['success']} | Allergens: {list(resp['detected_allergens'].keys())}")
    print(f"Avg confidence: {resp['avg_confidence']:.2f}")
    print(f"Error: {resp['error']}")

## Section F: Latency and Throughput

In [ ]:
# Step 11: Compute latency stats

components = ['ocr', 'cleaning', 'ner', 'mapping', 'total']
latency = {c: [] for c in components}

for resp in ui_responses:
    if resp.get('timings'):
        for c in components:
            if c == 'total':
                latency[c].append(resp['timings'].get('total', sum(resp['timings'].values())))
            else:
                latency[c].append(resp['timings'].get(c, 0))

print("Latency summary (seconds):")
for c in components:
    if latency[c]:
        arr = np.array(latency[c])
        print(f"  {c:<8} | avg: {arr.mean():.3f} | p95: {np.percentile(arr,95):.3f} | max: {arr.max():.3f}")

if latency['total']:
    avg_total = np.mean(latency['total'])
    throughput = 1.0 / avg_total if avg_total > 0 else 0
    print(f"\nThroughput: {throughput:.2f} images/sec ({throughput*60:.1f} per minute)")

## Section G: Save Responses (Optional)

In [ ]:
# Step 12: Save UI responses to JSON
from datetime import datetime
output_dir = RESULTS_DIR / "ui_testing"
output_dir.mkdir(parents=True, exist_ok=True)

stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
ui_path = output_dir / f"ui_responses_{stamp}.json"

with open(ui_path, 'w', encoding='utf-8') as f:
    json.dump(ui_responses, f, indent=2, ensure_ascii=False)

print(f"✓ Saved {len(ui_responses)} responses to {ui_path}")

## Summary and Next Steps

- Validated API-style responses for allergen detection
- Collected latency stats for UI performance budgets
- Saved sample payloads for frontend integration testing

**Next:**
1) Integrate with frontend / backend service
2) Add retries or fallbacks for OCR failures
3) Tune confidence thresholds based on UI feedback
4) Expand test set and run batch mode if needed